#### Power iterations for `dominant` eigenvalue

Assume $A\in \mathbf{R}^{n \times n}$ is `diagonalizable`, we know its eigenvectors $v_i, \cdots, v_n$ form a `basis` for $\mathbf{R}^n$

For a vector $x\in \mathbf{R}^n$, we can express it using the eigenvectors of $A$ as

$$x=c_1v_1 + c_2v_2 + \cdots + c_nv_n$$

If we compute $Ax$ and assume $|\lambda_1|>|\lambda_2|\geq|\lambda_3| \cdots \geq |\lambda_n|$, we can write

$$\begin{align*}
Ax&=A(c_1v_1 + c_2v_2 + \cdots + c_nv_n) \\
&=c_1\lambda_1v_1 + c_2\lambda_2v_2+\cdots + c_n\lambda_nv_n
\end{align*}$$

If we keep multiplying A on the left, we get

$$\begin{align*}
A^kx&=A(c_1v_1 + c_2v_2 + \cdots + c_nv_n) \\
&=c_1\lambda_1^kv_1 + c_2\lambda_2^kv_2+\cdots + c_n\lambda_n^kv_n \\
&=\lambda_1^k\left(c_1v_1+c_2\left(\frac{\lambda_2}{\lambda_1}\right)^kv_2 + \cdots + c_n\left(\frac{\lambda_n}{\lambda_1}\right)^kv_n\right) \\
& k \rightarrow \infty, \left(\frac{\lambda_i}{\lambda_1}\right)\rightarrow 0, i\neq 1 \\
&=\lambda_1^kc_1v_1
\end{align*}$$

It provides an idea to compute the `dominant eigenvalue`

Apparently, we also need to normalize the process, otherwise the norm of the vector after many iterations goes to infinity or zero

* starting from $x^k$
* compute $y^k=Ax^k$
* get new $x^{k+1}$ by normalizing $y^k$ ($l_2$ norm, infinity norm, etc)
$$x^{k+1}=\frac{y^k}{\|y^k\|}$$

We can see that $x^{k}$ is some scalar multiple of $A^{k}x^0$, which in turn is some scalar multiple of $v_1$, with estimation error dominated by $\left(\frac{\lambda_2}{\lambda_1}\right)^k$

If we can get $x^k\rightarrow cv_1$, then eigenvalue $\lambda_1$ is simply obtained by computing $y^k = Ax^k \rightarrow \lambda_1cv_1$, or (which is Rayleigh quotient)

$$\frac{y^k\cdot x^k}{x^k\cdot x^k}\rightarrow\lambda_1$$

Example

$$A=\begin{bmatrix}8 & 3 \\2 &7\end{bmatrix}, x^0=\begin{bmatrix}1 \\ 1\end{bmatrix}$$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
A = np.array([[8., 3.], [2., 7.]]) # diagonalizable
x = np.array([1., 1.])

eigenvalues, eigenvectors = np.linalg.eig(A)
print(f'True eigenvalues: {eigenvalues}')
print(f'True eigenvectors (columns): {eigenvectors}\n')

num_iter = 20
for iter in range(num_iter):
    y = A @ x

    # Rayleigh quotient
    lambda_1 = np.dot(y, x) / np.dot(x, x)
    print(f'# {iter+1}: lambda_1: {lambda_1:.4f}')

    x= y / np.linalg.norm(y)
v_1 = A @ x / np.linalg.norm(A @ x)
print(f'\nv_1: {v_1}')

True eigenvalues: [ 10.0000  5.0000]
True eigenvectors (columns): [[ 0.8321 -0.7071]
 [ 0.5547  0.7071]]

# 1: lambda_1: 10.0000
# 2: lambda_1: 10.0495
# 3: lambda_1: 10.0367
# 4: lambda_1: 10.0212
# 5: lambda_1: 10.0113
# 6: lambda_1: 10.0058
# 7: lambda_1: 10.0030
# 8: lambda_1: 10.0015
# 9: lambda_1: 10.0007
# 10: lambda_1: 10.0004
# 11: lambda_1: 10.0002
# 12: lambda_1: 10.0001
# 13: lambda_1: 10.0000
# 14: lambda_1: 10.0000
# 15: lambda_1: 10.0000
# 16: lambda_1: 10.0000
# 17: lambda_1: 10.0000
# 18: lambda_1: 10.0000
# 19: lambda_1: 10.0000
# 20: lambda_1: 10.0000

v_1: [ 0.8321  0.5547]


#### Iteration for other eigenvalues

After computing the dominant eigenvalue and its corresponding eigenvector, we proceed by updating the matrix $A$ through

$$A\leftarrow A - \lambda_1 v_1 v_1^T$$

This process is repeated iteratively to find more eigenvalues and eigenvectors

However, this approach works reliably `only for symmetric matrices`, due to the `orthogonality` of their eigenvectors

In symmetric matrices, all eigenvectors corresponding to distinct eigenvalues are orthogonal

When we subtract a term involving $v_1 v_1^T$, we effectively remove the contribution of $v_1$ from the matrix

The orthogonality ensures that removing $v_1$'s influence doesn't interfere with the subsequent eigenvectors, allowing the power iterations to correctly converge to the next dominant eigenvalue and eigenvector

In [ ]:
def power_iteration(A_sym, num_eigen, num_iter=2000, tol=1e-6):
    n = A_sym.shape[0]
    eigenvalues = []
    eigenvectors = []
    A_current = A_sym.copy()

    for k in range(num_eigen):
        x_k = np.random.rand(n)

        for j in range(num_iter):
            y_k = A_current @ x_k
            y_k_unit = y_k / np.linalg.norm(y_k)

            if np.linalg.norm(y_k_unit - x_k) < tol:
                break

            x_k = y_k_unit

        print(f'{j+1} iterations for eigenvalue #{k+1}')

        # Rayleigh quotient
        eigenvalue = np.dot(A_sym @ x_k, x_k) / np.dot(x_k, x_k)
        eigenvalues.append(eigenvalue)
        eigenvectors.append(x_k)

        A_current -= eigenvalue * np.outer(y_k_unit, y_k_unit) / np.dot(y_k_unit, y_k_unit)

    return np.array(eigenvalues), np.column_stack(eigenvectors)

In [ ]:
np.random.seed(42)

k = 5

A = np.random.rand(5, 5)

A_sym = (A + A.T) / 2

eigenvalues, eigenvectors = power_iteration(A_sym, k)

print("\nComputed eigenvalues:")
for idx, eigenvalue in enumerate(eigenvalues, 1):
    print(f"# {idx}: {eigenvalue:.4f}")

print("\nComputed eigenvectors (columns):")
print(eigenvectors)

true_eigenvalues, true_eigenvectors = np.linalg.eig(A_sym)
print("\nEigenvalues from NumPy:")
print(true_eigenvalues)

print("\nEigenvectors from NumPy:")
print(true_eigenvectors)

12 iterations for eigenvalue #1
2000 iterations for eigenvalue #2
10 iterations for eigenvalue #3
8 iterations for eigenvalue #4
2000 iterations for eigenvalue #5

Computed eigenvalues:
# 1: 2.2645
# 2: -0.6570
# 3: 0.4611
# 4: 0.0990
# 5: -0.0145

Computed eigenvectors (columns):
[[ 0.4067  0.2198  0.3005  0.0428  0.8332]
 [ 0.4865 -0.8603 -0.1312 -0.0666  0.0402]
 [ 0.5749  0.4306 -0.6653 -0.1404 -0.1471]
 [ 0.3846  0.0929  0.3070  0.7856 -0.3633]
 [ 0.3456  0.1326  0.5963 -0.5973 -0.3881]]

Eigenvalues from NumPy:
[ 2.2645 -0.6570  0.4611 -0.0145  0.0990]

Eigenvectors from NumPy:
[[-0.4067 -0.2198  0.3005  0.8332 -0.0428]
 [-0.4865  0.8603 -0.1312  0.0402  0.0666]
 [-0.5749 -0.4306 -0.6653 -0.1471  0.1404]
 [-0.3846 -0.0929  0.3070 -0.3633 -0.7856]
 [-0.3456 -0.1326  0.5963 -0.3881  0.5973]]
